In [1]:
from bs4 import BeautifulSoup
import requests 
import pandas as pd 

headers = {'User Agent':'Mozilla/5.0'}

players = [
    'deandre-ayton', 'marvin-bagleyiii', 'jaren-jacksonjr', 'trae-young', 'mo-bamba', 'wendell-carterjr', 
    'collin-sexton','kevin-knox', 'mikal-bridges', 'shai-gilgeous-alexander', 'miles-bridges', 'jerome-robinson', 
    'michael-porterjr', 'troy-brownjr', 'zhaire-smith', 'donte-divincenzo', 'lonnie-walkeriv', 'kevin-huerter', 
    'josh-okogie', 'grayson-allen','chandler-hutchinson', 'aaron-holiday', 'moritz-wagner', 'landry-shamet', 
    'robert-williams', 'jacob-evans', 'omari-spellman', 'jevon-carter', 'jalen-brunson', 'devonte-graham', 
    'gary-trentjr', 'khyri-thomas', 'jarred-vanderbilt', 'bruce-brown','hamidou-diallo', 'deanthony-melton', 
    'sviatoslav-mykhailiuk', 'keita-bates-diop', 'chimezie-metu', 'alize-johnson', 'tony-carr', 'vince-edwards', 
    'devon-hall', 'malik-milton', 'ray-spalding', 'thomas-welsh', 'george-king', 'kostas-antetokounmpo']

player_stats = []
playerlist = []
year_list = []
age_list = []

for player in players:
    try:
        if player[-1].isdigit():
            url = f'https://www.sports-reference.com/cbb/players/{player}.html'
        else:
            url = f'https://www.sports-reference.com/cbb/players/{player}-1.html'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        header = [th.getText() for th in soup.findAll('tr', limit = 2)[0].findAll('th')]
        rows = soup.findAll('tr')
        player_stats.append([td.getText() for td in soup.find('tr', id ='players_per_game.2018')])
        table = pd.DataFrame(player_stats, columns = header) 
        for i in player_stats:
            if i[14] == '':
                i[14] = str(0.0)
        
        player = player.replace('-',' ').title()
        if player[-1].isdigit():
            player = player[:-2]
        if player[-2:] == 'jr':
            player = player.replace('jr', ' Jr')
        if player[-3:] == 'iii':
            player = player.replace('iii', ' III')
        playerlist.append(player)
    except:
        continue
    try:
        url = f'https://www.google.com/search?q={player}+basketball+birth+date'
        res = requests.get(url)
        soup = BeautifulSoup(res.content, 'lxml')
        age = (soup.findAll('div', class_ = 'BNeawe iBp4i AP7Wnd')[0])
        for x in age:
            x = x.text
        x = x.replace(',','').split()
        months = {
            'Janurary': '1',
            'February': '2',
            'March': '3',
            'April': '4',
            'May': '5',
            'June': '6',
            'July': '7',
            'August': '8',
            'September': '9',
            'October': '10',
            'November': '11',
            'December': '12'
        } 
        birth_date = months[x[0]]+ '-' + x[1] + '-' + x[2]
        draft_date = '7-1-2018'
        import datetime 
        birth_date_final = datetime.datetime.strptime(birth_date, '%m-%d-%Y')  
        draft_date_final = datetime.datetime.strptime(draft_date, '%m-%d-%Y')

        age_on_draft_night = draft_date_final - birth_date_final
        age = round(age_on_draft_night.days/365, 1)
        age_list.append(age)
        year = round(((1-(age/30))*2.95), 2)
        year_list.append(year)
        
    except:
        age_list.append(22)
        year_list.append(.85)


table.insert(0, "Name", playerlist)
table.insert(2, "Year", year_list)
table.insert(2, "Age", age_list)
table['MP'] = table['MP'].astype(float)
table['PTS'] = table['PTS'].astype(float)
table['AST'] = table['AST'].astype(float)
table['TRB'] = table['TRB'].astype(float)
table['BLK'] = table['BLK'].astype(float)
table['STL'] = table['STL'].astype(float)
table['3P'] = table['3P'].astype(float)
table['3PA'] = table['3PA'].astype(float)
table['TOV'] = table['TOV'].astype(float)
table['SOS'] = table['SOS'].astype(float)
table['PF'] = table['PF'].astype(float)
table['FT%'] = table['FT%'].astype(float)
table['3P%'] = table['3P%'].astype(float)

per40 = 40/table['MP']

table["Player Grade"] = ((table['PTS']*per40) + (table['TRB']*1.5*per40) + (table['AST']*2*per40) +
(table['BLK']*3*per40) + (table['STL']*3*per40) + (table['3P']*5*per40)+ (table['FT%']*7) + (table['SOS']) + 
(table['3P%']*10)) * table['Year']

table["Player Grade"] = table["Player Grade"]*1.25
table["Player Grade"] = (round(table["Player Grade"], 1))
table["Player Grade"]= table["Player Grade"].astype(float)
                        
table = table.sort_values(by= "Player Grade", ascending=False)
table.insert(5, 'SoS', table['SOS'])
table.reset_index(drop = True, inplace=True)
import numpy as np
table.index = np.arange(1, len(table)+1)
del table['SOS']
del table['Conf']
del table['ORB']
del table['DRB']
del table['GS']
del table['FG']
del table['FGA']
del table['FT']
del table['FTA']
del table['2P']
del table['2PA']
del table['2P%']
del table['Year']
del table['Age']
table

,Name,Season,School,SoS,G,MP,FG%,3P,3PA,3P%,FT%,TRB,AST,STL,BLK,TOV,PF,PTS,,Player Grade
1,Trae Young,2017-18,Oklahoma,10.56,32,35.4,.422,3.7,10.3,0.360,0.861,3.9,8.7,1.7,0.3,5.2,1.8,27.4,,131.4
2,Jaren Jackson Jr,2017-18,Michigan State,7.10,35,21.8,.513,1.1,2.7,0.396,0.797,5.8,1.1,0.6,3.0,1.8,3.2,10.9,,119.0
3,Wendell Carter Jr,2017-18,Duke,9.71,37,26.9,.561,0.5,1.2,0.413,0.738,9.1,2.0,0.8,2.1,2.0,2.8,13.5,,108.6
4,Michael Porter Jr,2017-18,Missouri,9.19,3,17.7,.333,1.0,3.3,0.300,0.778,6.7,0.3,1.0,0.3,1.0,2.0,10.0,,103.4
5,Marvin Bagley III,2017-18,Duke,9.71,33,33.9,.614,0.7,1.8,0.397,0.627,11.1,1.5,0.8,0.9,2.3,1.8,21.0,,100.0
6,Jarred Vanderbilt,2017-18,Kentucky,10.73,14,17.0,.426,0.0,0.1,0.000,0.632,7.9,1.0,0.4,0.8,1.1,2.6,5.9,,92.0
7,Miles Bridges,2017-18,Michigan State,7.10,34,31.4,.457,2.1,5.7,0.364,0.853,7.0,2.7,0.6,0.8,2.0,1.6,17.1,,92.0
8,Deandre Ayton,2017-18,Arizona,6.84,35,33.5,.612,0.3,1.0,0.343,0.733,11.6,1.6,0.6,1.9,2.0,2.3,20.1,,91.6
9,Kevin Knox,2017-18,Kentucky,10.73,37,32.4,.445,1.5,4.5,0.341,0.774,5.4,1.4,0.8,0.3,2.3,2.2,15.6,,89.4
10,Josh Okogie,2017-18,Georgia Tech,7.47,24,36.4,.416,1.6,4.2,0.380,0.821,6.3,2.5,1.8,1.0,2.5,2.9,18.2,,88.6
